In [1]:
import os
import json
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from IPython.display import Markdown, display, HTML

os.chdir(os.path.dirname(os.getcwd()))

In [2]:
DEFAULT_PROMPT_ROOT_DIR = "omni/prompts"
DEFAULT_PROMPT_FILE = "prompt_concierge.txt"
DEFAULT_TOPIC_DIR = "cons"
DEFAULT_TOPIC = "Legal Services Department Inquiries"

In [3]:
PROMPT_PATH = Path(DEFAULT_PROMPT_ROOT_DIR) / DEFAULT_PROMPT_FILE

PREVIOUS_COMPLETIONS_PATH = (
    Path(DEFAULT_PROMPT_ROOT_DIR)
    / "knowledge_bases"
    / DEFAULT_TOPIC_DIR
    / "previous_completions.json"
)
DOMAIN_KNOWLEDGE_PATH = (
    Path(DEFAULT_PROMPT_ROOT_DIR)
    / "knowledge_bases"
    / DEFAULT_TOPIC_DIR
    / "domain_knowledge.txt"
)

with PROMPT_PATH.open("r") as file:
    prompt = file.read()

with PREVIOUS_COMPLETIONS_PATH.open("r") as file:
    previous_completions = file.read()

with DOMAIN_KNOWLEDGE_PATH.open("r") as file:
    domain_knowledge = file.read()

In [4]:
print(domain_knowledge)

# Employment Disputes
Handles legal issues concerning current or former employees pursuing legal action against the company.
Cases include wrongful termination, payment-related disputes, or contractual matters.
Does not handle discrimination accusations.

# Coverage Team
Specialized attorneys providing advice on complex claims.
Assist with policy definitions and their application.
Conduct case law research.
Referrals are only for non-litigated matters.
For litigation, refer to Field Legal.

# Field Legal
Manages all litigation matters.
Handles cases referred from the Coverage Team and other departments.
Specializes in defending the company in court.

# Compliance Team
Ensures company adherence to legal standards and regulations.
Handles internal audits and regulatory reviews.
Provides guidance on compliance-related issues.

# Fraud Investigation Unit
Investigates suspected insurance fraud cases.
Works closely with law enforcement and other agencies.
Provides support for both internal a

In [5]:
import re
from typing import List


def extract_class_labels(input_text: str) -> List[str]:
    """
    Extracts a list of class labels from an input string containing class label descriptions.

    Args:
        input_text (str): A string containing class label descriptions with h1 header tags.

    Returns:
        List[str]: A list of distinct class labels extracted from the input text.

    Example:
        >>> text = "# Employment Disputes\nHandles legal issues...\n# Coverage Team\nSpecialized attorneys..."
        >>> extract_class_labels(text)
        ['Employment Disputes', 'Coverage Team']
    """
    import re

    # Use regex to find all occurrences of '# ' followed by any characters until the end of the line
    pattern = r'^# (.+)$'
    
    # Find all matches in the input text, considering multiline mode
    matches = re.findall(pattern, input_text, re.MULTILINE)
    
    # Return the list of matched class labels
    return matches


def parse_class_descriptions(input_text: str) -> List[str]:
    """
    Extracts class descriptions from a Markdown-formatted text.

    Args:
        input_text (str): A string containing class descriptions with h1 header tags.

    Returns:
        List[str]: A list of strings, each containing a header and its description.

    Example:
        >>> text = "# Employment Disputes\nHandles legal issues...\n# Coverage Team\nSpecialized attorneys..."
        >>> parse_class_descriptions(text)
        ['Employment Disputes:\nHandles legal issues...', 'Coverage Team:\nSpecialized attorneys...']
    """
    pattern = r'#\s*(.+?)\n((?:(?!#\s).)+\n?)*'
    matches = re.findall(pattern, input_text, re.DOTALL)
    return [f"{header.strip()}:\n{description.strip()}" for header, description in matches]

In [6]:
class_labels = extract_class_labels(domain_knowledge)
class_descriptions = parse_class_descriptions(domain_knowledge)
len(class_labels), len(class_descriptions)

(139, 139)

In [7]:
df = pd.DataFrame({
    "department": class_labels,
    "context": class_descriptions,
})

In [8]:
import dspy
from src.agent.tools.splade_search import SPLADESparseSearch

gpt4o = dspy.OpenAI(model='gpt-4o')

In [42]:
from typing import List, Union, Optional

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


class DSPySPLADE(dspy.Retrieve):
    def __init__(self, df: pd.DataFrame, k:int = 7):
        super().__init__(k=k)

        self.splade = SPLADESparseSearch(
            df=df,
            text_column="context",
        )

    def forward(self, user_query:str, k:Optional[int] = None) -> dspy.Prediction:
        params = {"user_query": user_query, "k": k if k else self.k}
        response = self.splade.query_similar_documents(
            query=params["user_query"],
            top_k=params["k"],
        )
        context = response["context"].tolist()   # List of top k passages
        return [dotdict({"long_text": passage}) for passage in context]

In [43]:
splade_rm = DSPySPLADE(df=df)

2024-06-30 20:38:18 - INFO - Using pre-computed 'context' embeddings from existing column: splade_embeddings


In [44]:
test = splade_rm.forward(user_query="Who can help me with a NDA dispute?")
test

[{'long_text': 'Contractual Disputes Team:\nResolves disputes arising from contracts with vendors, partners, and clients.\nProvides negotiation and litigation support.\nAdvises on contract drafting and risk management.'},
 {'long_text': 'Alternative Dispute Resolution Team:\nProvides support for alternative dispute resolution methods.\nHandles mediation, arbitration, and other non-litigation dispute resolution processes.\nWorks with other departments to develop ADR strategies and initiatives.'},
 {'long_text': 'Bad Faith Allegations Team:\nHandles claims and disputes involving allegations of bad faith.\nProvides legal defense and advice on bad faith claims.\nWorks with claims adjusters and other departments to address bad faith issues.'},
 {'long_text': 'Employment Disputes:\nHandles legal issues concerning current or former employees pursuing legal action against the company.\nCases include wrongful termination, payment-related disputes, or contractual matters.\nDoes not handle discri

In [45]:
dspy.configure(lm=gpt4o, rm=splade_rm)

In [46]:
training_data = json.loads(previous_completions)
len(training_data)

37

In [93]:
all_examples = []

for sample in training_data:
    example = dspy.Example(
        user_query=sample["user_query"],
        department=sample["department"],
        answer=sample["department"],
    ).with_inputs("user_query", "context")
    all_examples.append(example)

In [94]:
trainset = all_examples[:20]
devset = all_examples[20:35]
testset = all_examples[35:]

In [49]:
devset[0]

Example({'user_query': 'we need help with a patent infringement case', 'department': 'Intellectual Property Group'}) (input_keys={'context', 'user_query'})

In [97]:
# This is a WIP, the next step is to optimize this metric as itself a DSPy module (pretty meta)

# Reference - https://github.com/stanfordnlp/dspy/blob/main/examples/tweets/tweet_metric.py

metricLM = dspy.OpenAI(model='gpt-4-turbo', max_tokens=1000, model_type='chat')

# Signature for LLM assessments.

class Assess(dspy.Signature):
    """Assess the quality of an answer to a question."""
    
    context = dspy.InputField(desc="The context for answering the question.")
    assessed_question = dspy.InputField(desc="The evaluation criterion.")
    assessed_answer = dspy.InputField(desc="The answer to the question.")
    assessment_answer = dspy.OutputField(desc="A rating between 1 and 5. Only output the rating and nothing else.")

def llm_metric(gold, pred, trace=None):
    predicted_answer = pred.answer
    question = gold.user_query
    
    print(f"Test Question: {question}")
    print(f"Predicted Answer: predicted_answer")
    
    detail = "Is the assessed answer detailed?"
    faithful = "Is the assessed text grounded in the context? Say no if it includes significant facts not in the context."
    overall = f"Please rate how well this answer answers the question, `{question}` based on the context.\n `{predicted_answer}`"
    
    with dspy.context(lm=metricLM):
        context = dspy.Retrieve(k=10)(question).passages
        detail = dspy.ChainOfThought(Assess)(context="N/A", assessed_question=detail, assessed_answer=predicted_answer)
        faithful = dspy.ChainOfThought(Assess)(context=context, assessed_question=faithful, assessed_answer=predicted_answer)
        overall = dspy.ChainOfThought(Assess)(context=context, assessed_question=overall, assessed_answer=predicted_answer)
    
    print(f"Faithful: {faithful.assessment_answer}")
    print(f"Detail: {detail.assessment_answer}")
    print(f"Overall: {overall.assessment_answer}")
    
    
    total = float(detail.assessment_answer) + float(faithful.assessment_answer)*2 + float(overall.assessment_answer)
    
    return total / 5.0

In [153]:
class PredictionSignature(dspy.Signature):
    """Predict the best department to help with a new user query based on the context."""
    
    context = dspy.InputField(desc="may contain relevant facts")
    user_query = dspy.InputField()
    department = dspy.OutputField()

In [154]:
class ConciergeBot(dspy.Module):
    def __init__(self, num_passages=5):
        super().__init__()
        
        self.signature = PredictionSignature
        self.rm = dspy.Retrieve(k=num_passages)
        self.generate_prediction = dspy.ChainOfThought(self.signature)

    def forward(self, user_query):
        context = self.rm(user_query).passages
        result = self.generate_prediction(context=context, user_query=user_query)
        return dspy.Prediction(
            answer=result.department,
            reasoning=result.rationale,
        )

In [155]:
from dspy.evaluate import Evaluate

def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    return answer_EM

NUM_THREADS = 5
evaluate = Evaluate(devset=devset, metric=validate_context_and_answer, num_threads=NUM_THREADS, display_progress=True, display_table=False)

In [156]:
bot_baseline = ConciergeBot()

evaluate(bot_baseline, devset=devset)

Average Metric: 11 / 15  (73.3): 100%|██████████| 15/15 [00:03<00:00,  4.30it/s]


73.33

In [158]:
from dspy.teleprompt import COPRO

teleprompter = COPRO(
    metric=validate_context_and_answer,
)

In [159]:
kwargs = dict(num_threads=64, display_progress=True, display_table=5) # Used in Evaluate class in the optimization process

compiled_prompt_opt = teleprompter.compile(bot_baseline, trainset=devset, eval_kwargs=kwargs)

Average Metric: 3 / 15  (20.0): 100%|██████████| 15/15 [00:06<00:00,  2.45it/s]


,user_query,department,example_answer,pred_answer,reasoning,validate_context_and_answer
0,we need help with a patent infringement case,Intellectual Property Group,Intellectual Property Group,query_redirection Intellectual Property Group,"produce the department. We need assistance with a patent infringement case, which falls under the Intellectual Property Group's expertise. They handle cases involving IP rights,...",False
1,we're dealing with a class action lawsuit,Class Action Defense Team,Class Action Defense Team,query_redirection Class Action Litigation Team or Class Action Defense Team,"determine the best department to assist with a class action lawsuit. Since the user is specifically dealing with a class action lawsuit, the most suitable...",False
2,we need assistance with a corporate restructuring,Corporate Transactions Legal Team,Corporate Transactions Legal Team,Corporate Transactions Legal Team,"produce the department that can best assist with corporate restructuring. Given that corporate restructuring involves significant changes within a company's structure, operations, and finances, the...",✔️ [True]
3,we're facing a crisis situation and need legal support,Crisis Management Legal Team,Crisis Management Legal Team,query_redirection Crisis Management Legal Team or Crisis Response Legal Team,"produce the department that can assist with a crisis situation and provide legal support. Given the nature of the crisis, the best department to assist...",False
4,we need help with e-discovery for an upcoming case,E-Discovery Legal Team,E-Discovery Legal Team,query_redirection E-Discovery Legal Team or E-Discovery Management Team,"produce the department. We need assistance with e-discovery for an upcoming case, which involves managing electronic discovery processes, data collection, review, and production. This aligns...",False


Average Metric: 10 / 15  (66.7): 100%|██████████| 15/15 [00:02<00:00,  6.85it/s]


,user_query,department,example_answer,pred_answer,reasoning,validate_context_and_answer
0,we need help with a patent infringement case,Intellectual Property Group,Intellectual Property Group,Intellectual Property Group,"produce the department. We need a team that specializes in handling cases involving IP rights, specifically patents. This team should be able to provide defense...",✔️ [True]
1,we're dealing with a class action lawsuit,Class Action Defense Team,Class Action Defense Team,Class Action Defense Team,"produce the department that can best handle a class action lawsuit. First, we need a team that specializes in providing defense against class action lawsuits...",✔️ [True]
2,we need assistance with a corporate restructuring,Corporate Transactions Legal Team,Corporate Transactions Legal Team,Mergers & Acquisitions Legal Team,"produce the department. We need legal expertise in corporate transactions, deals, mergers, acquisitions, and potentially disputes related to corporate restructuring. The department that best fits...",False
3,we're facing a crisis situation and need legal support,Crisis Management Legal Team,Crisis Management Legal Team,Crisis Response Legal Team,"produce the department that can best address the crisis situation and provide legal support. Considering the expertise and focus of the departments listed, both the...",False
4,we need help with e-discovery for an upcoming case,E-Discovery Legal Team,E-Discovery Legal Team,E-Discovery Legal Team or E-Discovery Management Team,"produce the department. We need a team that specializes in managing electronic discovery processes for litigation matters, providing support for data collection, review, and production....",False


Average Metric: 11 / 15  (73.3): 100%|██████████| 15/15 [00:02<00:00,  7.39it/s]


...

,user_query,department,example_answer,pred_answer,reasoning,validate_context_and_answer
0,we need help with a patent infringement case,Intellectual Property Group,Intellectual Property Group,Intellectual Property Group,"produce the department best equipped to handle the user query. Given that the user query involves a patent infringement case, the most suitable department to...",✔️ [True]
1,we're dealing with a class action lawsuit,Class Action Defense Team,Class Action Defense Team,Class Action Litigation Team or Class Action Defense Team,produce the department best equipped to handle the query effectively and efficiently. Detailed Department Analysis: Given that the user query involves dealing with a class...,False
2,we need assistance with a corporate restructuring,Corporate Transactions Legal Team,Corporate Transactions Legal Team,Mergers & Acquisitions Legal Team,produce the department best equipped to handle the query effectively and efficiently. Detailed Department Analysis: Mergers & Acquisitions Legal Team The user query pertains to...,False
3,we're facing a crisis situation and need legal support,Crisis Management Legal Team,Crisis Management Legal Team,Crisis Management Legal Team or Crisis Response Legal Team,"produce the department best equipped to handle the user query. Given that the user is facing a crisis situation and requires legal support, the most...",False
4,we need help with e-discovery for an upcoming case,E-Discovery Legal Team,E-Discovery Legal Team,E-Discovery Legal Team or E-Discovery Management Team The E-Discovery Legal Team and E-Discovery Management Team are both well-equipped to handle e-discovery processes for litigation matters....,determine the department best equipped to handle the user query effectively and efficiently. Given that the user query specifically mentions needing help with e-discovery for...,False


Average Metric: 12 / 15  (80.0): 100%|██████████| 15/15 [00:02<00:00,  6.54it/s] 


,user_query,department,example_answer,pred_answer,reasoning,validate_context_and_answer
0,we need help with a patent infringement case,Intellectual Property Group,Intellectual Property Group,Intellectual Property Group,"produce the department. We can utilize natural language processing to analyze the user query ""we need help with a patent infringement case"" and historical data...",✔️ [True]
1,we're dealing with a class action lawsuit,Class Action Defense Team,Class Action Defense Team,Class Action Defense Team,"produce the department. We need to address the class action lawsuit efficiently and effectively. Since the user query specifically mentions a class action lawsuit, the...",✔️ [True]
2,we need assistance with a corporate restructuring,Corporate Transactions Legal Team,Corporate Transactions Legal Team,Corporate Transactions Legal Team,"produce the department. We can see that the user query involves a corporate restructuring, which typically involves legal matters related to corporate transactions, mergers, acquisitions,...",✔️ [True]
3,we're facing a crisis situation and need legal support,Crisis Management Legal Team,Crisis Management Legal Team,Crisis Management Legal Team,"produce the department. We can utilize natural language processing to analyze the user query and historical data. Based on the context provided, it seems like...",✔️ [True]
4,we need help with e-discovery for an upcoming case,E-Discovery Legal Team,E-Discovery Legal Team,E-Discovery Legal Team or E-Discovery Management Team,produce the department. We should recommend the E-Discovery Legal Team or the E-Discovery Management Team. Both teams specialize in managing electronic discovery processes for litigation...,False


Average Metric: 13 / 15  (86.7): 100%|██████████| 15/15 [00:03<00:00,  4.26it/s]


,user_query,department,example_answer,pred_answer,reasoning,validate_context_and_answer
0,we need help with a patent infringement case,Intellectual Property Group,Intellectual Property Group,Intellectual Property Group,"produce the department. We need to consider the nature of the case, which involves intellectual property rights specifically related to patents. Given that the case...",✔️ [True]
1,we're dealing with a class action lawsuit,Class Action Defense Team,Class Action Defense Team,Class Action Defense Team,"determine the premium department for addressing a class action lawsuit query. Given that the user query involves a class action lawsuit, we need a department...",✔️ [True]
2,we need assistance with a corporate restructuring,Corporate Transactions Legal Team,Corporate Transactions Legal Team,Corporate Transactions Legal Team,"ascertain the premium department for addressing the query. 1. The user query mentions ""corporate restructuring,"" which typically involves significant changes in a company's structure, operations,...",✔️ [True]
3,we're facing a crisis situation and need legal support,Crisis Management Legal Team,Crisis Management Legal Team,Crisis Management Legal Team,"select the Crisis Management Legal Team. We need legal support for crisis management and response efforts, which is the primary focus of this department. They...",✔️ [True]
4,we need help with e-discovery for an upcoming case,E-Discovery Legal Team,E-Discovery Legal Team,E-Discovery Legal Team or E-Discovery Management Team,"select the premium department for addressing the query. Given that the user query specifically mentions needing help with e-discovery for an upcoming case, the most...",False


In [163]:
compiled_prompt_opt.candidate_programs[0]

{'score': 86.67,
 'program': generate_prediction = ChainOfThought(PredictionSignature(context, user_query -> department
     instructions='Predict the best department to help with a new user query based on the context.'
     context = Field(annotation=str required=True json_schema_extra={'desc': 'may contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
     user_query = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'User Query:', 'desc': '${user_query}'})
     department = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Department:', 'desc': '${department}'})
 )),
 'instruction': 'Instruction #11: Analyze the user query, taking into consideration contextual cues and historical context, to programmatically suggest the optimal department for addressing the query proficiently.',
 'prefix': 'optimized_department_suggestion',
 'depth': 2}

In [169]:
compiled_prompt_opt.dump_state()

{'rm': {'k': 5},
 'generate_prediction': {'lm': None,
  'traces': [],
  'train': [],
  'demos': [],
  'signature_instructions': 'Predict the best department to help with a new user query based on the context.',
  'signature_prefix': 'Department:',
  'extended_signature_instructions': 'Instruction #11: Analyze the user query, taking into consideration contextual cues and historical context, to programmatically suggest the optimal department for addressing the query proficiently.',
  'extended_signature_prefix': 'optimized_department_suggestion'}}

In [172]:
class OptimizedDepartmentSuggestion(dspy.Signature):
    """Analyze the user query, taking into consideration contextual cues and historical context, to 
programmatically suggest the optimal department for addressing the query proficiently."""
    
    context = dspy.InputField(desc="may contain relevant facts")
    user_query = dspy.InputField()
    department = dspy.OutputField()
    
    
class ConciergeBot(dspy.Module):
    def __init__(self, num_passages=5):
        super().__init__()
        
        self.signature = OptimizedDepartmentSuggestion
        self.rm = dspy.Retrieve(k=num_passages)
        self.generate_prediction = dspy.ChainOfThought(self.signature)

    def forward(self, user_query):
        context = self.rm(user_query).passages
        result = self.generate_prediction(context=context, user_query=user_query)
        return dspy.Prediction(
            answer=result.department,
            reasoning=result.rationale,
        )

In [171]:
gpt4o.inspect_history(n=1)




Predict the best department to help with a new user query based on the context.

---

User Query: our company is being sued for alleged discrimination
Department: Employment Disputes

User Query: we have a dispute with a vendor regarding a contract
Department: Contractual Disputes Team

User Query: we are dealing with a challenge related to intellectual property
Department: Intellectual Property Group

User Query: i have a legal case that requires representation in court
Department: Field Legal

---

Follow the following format.

Context: may contain relevant facts

User Query: ${user_query}

Reasoning: Let's think step by step in order to ${produce the department}. We ...

Department: ${department}

---

Context:
[1] «Employment Benefits Legal Team:
Handles legal issues related to employee benefits and compensation.
Provides advice on retirement plans, health benefits, and compliance.
Manages disputes and litigation involving employee benefits.»
[2] «Employment Disputes:
Handles le

"\n\n\nPredict the best department to help with a new user query based on the context.\n\n---\n\nUser Query: our company is being sued for alleged discrimination\nDepartment: Employment Disputes\n\nUser Query: we have a dispute with a vendor regarding a contract\nDepartment: Contractual Disputes Team\n\nUser Query: we are dealing with a challenge related to intellectual property\nDepartment: Intellectual Property Group\n\nUser Query: i have a legal case that requires representation in court\nDepartment: Field Legal\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nUser Query: ${user_query}\n\nReasoning: Let's think step by step in order to ${produce the department}. We ...\n\nDepartment: ${department}\n\n---\n\nContext:\n[1] «Employment Benefits Legal Team:\nHandles legal issues related to employee benefits and compensation.\nProvides advice on retirement plans, health benefits, and compliance.\nManages disputes and litigation involving employee benefits.

In [173]:
from dspy.evaluate.evaluate import Evaluate

evaluate = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5, return_outputs=False)

metric = dspy.evaluate.answer_exact_match

evaluate(ConciergeBot(), metric=metric)

Average Metric: 11 / 15  (73.3): 100%|██████████| 15/15 [00:17<00:00,  1.18s/it]


,user_query,department,example_answer,pred_answer,reasoning,answer_exact_match
0,we need help with a patent infringement case,Intellectual Property Group,Intellectual Property Group,Intellectual Property Group,"produce the department. Since the user query specifically mentions a patent infringement case, the optimal department to address this proficiently would be the Intellectual Property...",✔️ [True]
1,we're dealing with a class action lawsuit,Class Action Defense Team,Class Action Defense Team,Class Action Litigation Team,"produce the department. We need a team that specializes in handling class action lawsuits, providing legal defense, and working with other departments to address such...",False
2,we need assistance with a corporate restructuring,Corporate Transactions Legal Team,Corporate Transactions Legal Team,Mergers & Acquisitions Legal Team,"produce the department. We need to consider that corporate restructuring involves significant changes in the organization's structure, operations, and possibly legal matters related to mergers,...",False
3,we're facing a crisis situation and need legal support,Crisis Management Legal Team,Crisis Management Legal Team,Crisis Management Legal Team,"produce the department. We need legal support for crisis situations, which involves handling legal issues related to emergencies and developing crisis management plans. Given the...",✔️ [True]
4,we need help with e-discovery for an upcoming case,E-Discovery Legal Team,E-Discovery Legal Team,E-Discovery Legal Team,"produce the department. We need a team that specifically focuses on managing electronic discovery processes for litigation matters, providing support for data collection, review, and...",✔️ [True]


73.33

In [167]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

gpt4T = dspy.OpenAI(model='gpt-4o', max_tokens=350, model_type='chat')

bootstrap_optimizer = BootstrapFewShotWithRandomSearch(
    max_bootstrapped_demos=8,
    max_labeled_demos=8,
    num_candidate_programs=10,
    num_threads=8,
    metric=metric,
    teacher_settings=dict(lm=gpt4T))


cot_fewshot = bootstrap_optimizer.compile(ConciergeBot(), trainset=trainset, valset=devset)
# cot_fewshot.save("compiled_bot_cot_fewshot.json")

  0%|          | 0/15 [00:00<?, ?it/s]

Average Metric: 12 / 15  (80.0): 100%|██████████| 15/15 [00:04<00:00,  3.29it/s]


In [ ]:
cot_fewshot.save("compiled_bot_cot_fewshotv2.json")

In [176]:
evaluator = Evaluate(devset=testset, num_threads=1, display_progress=True, display_table=5)

evaluator(cot_fewshot, metric=metric)

Average Metric: 1 / 2  (50.0): 100%|██████████| 2/2 [00:04<00:00,  2.10s/it] 


,user_query,department,example_answer,pred_answer,reasoning,answer_exact_match
0,we're facing supply chain contract issues,Supply Chain Legal Team,Supply Chain Legal Team,Supply Chain Legal Team,": Analyze the user query, taking into consideration contextual cues and historical context, to programmatically suggest the optimal department for addressing the query proficiently.",✔️ [True]
1,we need help with a tax audit,Tax Compliance Legal Team,Tax Compliance Legal Team,Tax Legal Team,": Analyze the user query, taking into consideration contextual cues and historical context, to programmatically suggest the optimal department for addressing the query proficiently.",False


50.0

In [147]:
cot_fewshot.save("compiled_bot_cot_fewshot.json")

In [177]:
gpt4o.inspect_history(1)




Predict the best department to help with a new user query based on the context.

---

User Query: our company is being sued for alleged discrimination
Department: Employment Disputes

User Query: we have a dispute with a vendor regarding a contract
Department: Contractual Disputes Team

User Query: we are dealing with a challenge related to intellectual property
Department: Intellectual Property Group

User Query: i have a legal case that requires representation in court
Department: Field Legal

---

Follow the following format.

Context: may contain relevant facts

User Query: ${user_query}

Reasoning: Let's think step by step in order to ${produce the department}. We ...

Department: ${department}

---

Context:
[1] «Employment Benefits Legal Team:
Handles legal issues related to employee benefits and compensation.
Provides advice on retirement plans, health benefits, and compliance.
Manages disputes and litigation involving employee benefits.»
[2] «Employment Disputes:
Handles le

"\n\n\nPredict the best department to help with a new user query based on the context.\n\n---\n\nUser Query: our company is being sued for alleged discrimination\nDepartment: Employment Disputes\n\nUser Query: we have a dispute with a vendor regarding a contract\nDepartment: Contractual Disputes Team\n\nUser Query: we are dealing with a challenge related to intellectual property\nDepartment: Intellectual Property Group\n\nUser Query: i have a legal case that requires representation in court\nDepartment: Field Legal\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nUser Query: ${user_query}\n\nReasoning: Let's think step by step in order to ${produce the department}. We ...\n\nDepartment: ${department}\n\n---\n\nContext:\n[1] «Employment Benefits Legal Team:\nHandles legal issues related to employee benefits and compensation.\nProvides advice on retirement plans, health benefits, and compliance.\nManages disputes and litigation involving employee benefits.

In [125]:
from dspy.teleprompt import BootstrapFewShot

teleprompter = BootstrapFewShot(metric=metric, max_labeled_demos=8, max_rounds=3)

compiled_bot = teleprompter.compile(uncompiled_bot, trainset=trainset)

  0%|          | 0/20 [00:00<?, ?it/s]


In [126]:
evaluate(compiled_bot, metric=metric)

Average Metric: 13 / 15  (86.7): 100%|██████████| 15/15 [00:26<00:00,  1.77s/it]


,user_query,department,example_answer,pred_answer,answer_exact_match
0,we need help with a patent infringement case,Intellectual Property Group,Intellectual Property Group,Intellectual Property Group,✔️ [True]
1,we're dealing with a class action lawsuit,Class Action Defense Team,Class Action Defense Team,Class Action Litigation Team,False
2,we need assistance with a corporate restructuring,Corporate Transactions Legal Team,Corporate Transactions Legal Team,Corporate Transactions Legal Team,✔️ [True]
3,we're facing a crisis situation and need legal support,Crisis Management Legal Team,Crisis Management Legal Team,Crisis Management Legal Team,✔️ [True]
4,we need help with e-discovery for an upcoming case,E-Discovery Legal Team,E-Discovery Legal Team,E-Discovery Legal Team,✔️ [True]


(86.67,
 [(Example({'user_query': 'we need help with a patent infringement case', 'department': 'Intellectual Property Group', 'answer': 'Intellectual Property Group'}) (input_keys={'context', 'user_query'}),
   Prediction(
       answer='Intellectual Property Group'
   ),
   True),
  (Example({'user_query': "we're dealing with a class action lawsuit", 'department': 'Class Action Defense Team', 'answer': 'Class Action Defense Team'}) (input_keys={'context', 'user_query'}),
   Prediction(
       answer='Class Action Litigation Team'
   ),
   False),
  (Example({'user_query': 'we need assistance with a corporate restructuring', 'department': 'Corporate Transactions Legal Team', 'answer': 'Corporate Transactions Legal Team'}) (input_keys={'context', 'user_query'}),
   Prediction(
       answer='Corporate Transactions Legal Team'
   ),
   True),
  (Example({'user_query': "we're facing a crisis situation and need legal support", 'department': 'Crisis Management Legal Team', 'answer': 'Crisi

In [127]:
compiled_bot.dump_state()

{'rm': {'k': 5},
 'generate_prediction': {'lm': None,
  'traces': [],
  'train': [],
  'demos': [Example({'augmented': True, 'context': ['Employment Benefits Legal Team:\nHandles legal issues related to employee benefits and compensation.\nProvides advice on retirement plans, health benefits, and compliance.\nManages disputes and litigation involving employee benefits.', 'Employment Disputes:\nHandles legal issues concerning current or former employees pursuing legal action against the company.\nCases include wrongful termination, payment-related disputes, or contractual matters.\nDoes not handle discrimination accusations.', 'Workplace Safety Legal Team:\nHandles legal matters related to workplace safety and health regulations.\nProvides advice on compliance with workplace safety standards.\nManages disputes and litigation involving workplace safety issues.', 'Workplace Investigations Legal Team:\nHandles legal matters related to workplace investigations and misconduct.\nProvides advi

In [128]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

teleprompter = BootstrapFewShotWithRandomSearch(metric=metric, 
                                                max_bootstrapped_demos=4,
                                                max_labeled_demos=4, 
                                                max_rounds=1,
                                                num_candidate_programs=2,
                                                num_threads=2)

second_compiled_bot = teleprompter.compile(uncompiled_bot, trainset=trainset)

Average Metric: 14 / 20  (70.0): 100%|██████████| 20/20 [00:00<00:00, 36.30it/s]


In [129]:
gpt4o.inspect_history(n=1)




Predict the best department to help with a new user query based on the context.

---

User Query: our insured product has caused injuries and we need legal assistance
Department: Product Liability Team

User Query: i need assistance with a workers' compensation claim
Department: Workers' Compensation Legal Team

---

Follow the following format.

Context: may contain relevant facts

User Query: ${user_query}

Reasoning: Let's think step by step in order to ${produce the department}. We ...

Department: ${department}

---

Context:
[1] «Intellectual Property Group:
Handles cases involving IP rights, including patents, trademarks, and copyrights.
Provides defense against IP infringement claims.
Assists in securing and enforcing the companyâ€™s IP assets.»
[2] «Intellectual Property Transactions Team:
Handles legal matters related to IP transactions and licensing.
Provides advice on IP agreements and contracts.
Manages disputes and litigation involving IP transactions.»
[3] «Intellectu

'\n\n\nPredict the best department to help with a new user query based on the context.\n\n---\n\nUser Query: our insured product has caused injuries and we need legal assistance\nDepartment: Product Liability Team\n\nUser Query: i need assistance with a workers\' compensation claim\nDepartment: Workers\' Compensation Legal Team\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nUser Query: ${user_query}\n\nReasoning: Let\'s think step by step in order to ${produce the department}. We ...\n\nDepartment: ${department}\n\n---\n\nContext:\n[1] «Intellectual Property Group:\nHandles cases involving IP rights, including patents, trademarks, and copyrights.\nProvides defense against IP infringement claims.\nAssists in securing and enforcing the companyâ€™s IP assets.»\n[2] «Intellectual Property Transactions Team:\nHandles legal matters related to IP transactions and licensing.\nProvides advice on IP agreements and contracts.\nManages disputes and litigation invo

In [130]:
evaluate(second_compiled_bot, metric=metric)

Average Metric: 12 / 15  (80.0): 100%|██████████| 15/15 [00:24<00:00,  1.61s/it]


,user_query,department,example_answer,pred_answer,answer_exact_match
0,we need help with a patent infringement case,Intellectual Property Group,Intellectual Property Group,Intellectual Property Group,✔️ [True]
1,we're dealing with a class action lawsuit,Class Action Defense Team,Class Action Defense Team,Class Action Litigation Team,False
2,we need assistance with a corporate restructuring,Corporate Transactions Legal Team,Corporate Transactions Legal Team,Corporate Transactions Legal Team,✔️ [True]
3,we're facing a crisis situation and need legal support,Crisis Management Legal Team,Crisis Management Legal Team,Crisis Management Legal Team,✔️ [True]
4,we need help with e-discovery for an upcoming case,E-Discovery Legal Team,E-Discovery Legal Team,E-Discovery Legal Team,✔️ [True]


(80.0,
 [(Example({'user_query': 'we need help with a patent infringement case', 'department': 'Intellectual Property Group', 'answer': 'Intellectual Property Group'}) (input_keys={'context', 'user_query'}),
   Prediction(
       answer='Intellectual Property Group'
   ),
   True),
  (Example({'user_query': "we're dealing with a class action lawsuit", 'department': 'Class Action Defense Team', 'answer': 'Class Action Defense Team'}) (input_keys={'context', 'user_query'}),
   Prediction(
       answer='Class Action Litigation Team'
   ),
   False),
  (Example({'user_query': 'we need assistance with a corporate restructuring', 'department': 'Corporate Transactions Legal Team', 'answer': 'Corporate Transactions Legal Team'}) (input_keys={'context', 'user_query'}),
   Prediction(
       answer='Corporate Transactions Legal Team'
   ),
   True),
  (Example({'user_query': "we're facing a crisis situation and need legal support", 'department': 'Crisis Management Legal Team', 'answer': 'Crisis

In [131]:
uncompiled_bot.save('uncompiled_bot.json')

In [132]:
compiled_bot.save('compiled_bot.json')

In [133]:
second_compiled_bot.save('second_compiled_bot.json')

In [134]:
gpt4o.inspect_history(n=1)




Predict the best department to help with a new user query based on the context.

---

Follow the following format.

Context: may contain relevant facts

User Query: ${user_query}

Reasoning: Let's think step by step in order to ${produce the department}. We ...

Department: ${department}

---

Context:
[1] «Employment Benefits Legal Team:
Handles legal issues related to employee benefits and compensation.
Provides advice on retirement plans, health benefits, and compliance.
Manages disputes and litigation involving employee benefits.»
[2] «Employment Disputes:
Handles legal issues concerning current or former employees pursuing legal action against the company.
Cases include wrongful termination, payment-related disputes, or contractual matters.
Does not handle discrimination accusations.»
[3] «Workplace Safety Legal Team:
Handles legal matters related to workplace safety and health regulations.
Provides advice on compliance with workplace safety standards.
Manages disputes and liti

"\n\n\nPredict the best department to help with a new user query based on the context.\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nUser Query: ${user_query}\n\nReasoning: Let's think step by step in order to ${produce the department}. We ...\n\nDepartment: ${department}\n\n---\n\nContext:\n[1] «Employment Benefits Legal Team:\nHandles legal issues related to employee benefits and compensation.\nProvides advice on retirement plans, health benefits, and compliance.\nManages disputes and litigation involving employee benefits.»\n[2] «Employment Disputes:\nHandles legal issues concerning current or former employees pursuing legal action against the company.\nCases include wrongful termination, payment-related disputes, or contractual matters.\nDoes not handle discrimination accusations.»\n[3] «Workplace Safety Legal Team:\nHandles legal matters related to workplace safety and health regulations.\nProvides advice on compliance with workplace safety stand

In [178]:
from dspy.teleprompt import BayesianSignatureOptimizer

llm_prompter = dspy.OpenAI(model='gpt-4o', max_tokens=2000, model_type='chat')

teleprompter = BayesianSignatureOptimizer(task_model=dspy.settings.lm,
                                          metric=metric,
                                          prompt_model=llm_prompter,
                                          n=5,
                                          verbose=True)

kwargs = dict(num_threads=1, display_progress=True, display_table=0)

third_compiled_bot = teleprompter.compile(ConciergeBot(), devset=devset,
                                         optuna_trials_num=3,
                                         max_bootstrapped_demos=4,
                                         max_labeled_demos=4,
                                         eval_kwargs=kwargs)

third_compiled_bot.save('third_compiled_bot.json')

[WARNING] BayesianSignatureOptimizer has been deprecated and replaced with MIPRO.  BayesianSignatureOptimizer will be removed in a future release. 

Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Task Model: 15 examples in dev set * 3 trials * # of LM calls in your program = (45 * # of LM calls in your program) task model calls
- Prompt Model: # data summarizer calls (max 10) + 5 * 1 lm calls in program = 15 prompt model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calc

 40%|████      | 6/15 [00:06<00:09,  1.09s/it]


Creating basic bootstrap: 2/4


 27%|██▋       | 4/15 [00:04<00:13,  1.19s/it]


Creating basic bootstrap: 3/4


...

Starting trial #1
Evaling the following program:
Predictor 0
i: Proposed Instruction: Given the context describing the responsibilities and focus areas of various legal teams, classify the user query by matching it to the most appropriate legal department. Ensure that the classification leverages both the keywords and the specific legal issue mentioned within the query.
p: Recommended Department:


Average Metric: 12 / 15  (80.0): 100%|██████████| 15/15 [00:00<00:00, 28.74it/s]
0st split score: 80.0
curr average score: 80.0
Fully evaled score: 80.0
Model (<dsp.modules.gpt3.GPT3 object at 0x00000166DB773D90>) History:



Proposed Instruction: Given the context describing the responsibilities and focus areas of various legal teams, classify the user query by matching it to the most appropriate legal department. Ensure that the classification leverages both the keywords and the specific legal issue mentioned within the query.

---

Follow the following format.

Context: may contain releva

[I 2024-07-01 00:07:30,642] Trial 1 finished with value: 80.0 and parameters: {'1541423148256_predictor_instruction': 1, '1541423148256_predictor_demos': 2}. Best is trial 0 with value: 80.0.


Starting trial #2
Evaling the following program:
Predictor 0
i: Analyze the user query, taking into consideration contextual cues and historical context, to 
programmatically suggest the optimal department for addressing the query proficiently.
p: Department:


Average Metric: 11 / 15  (73.3): 100%|██████████| 15/15 [00:00<00:00, 28.63it/s]
0st split score: 73.33
curr average score: 73.33
Fully evaled score: 73.33
Model (<dsp.modules.gpt3.GPT3 object at 0x00000166DB773D90>) History:



Analyze the user query, taking into consideration contextual cues and historical context, to 
programmatically suggest the optimal department for addressing the query proficiently.

---

Follow the following format.

Context: may contain relevant facts

User Query: ${user_query}

Reasoning: Let's think step by step in order to ${produce the department}. We ...

Department: ${department}

---

Context:
[1] «Fraud Investigation Unit:
Investigates suspected insurance fraud cases.
Works closely with law enfo

[I 2024-07-01 00:07:31,191] Trial 2 finished with value: 73.33 and parameters: {'1541423148256_predictor_instruction': 0, '1541423148256_predictor_demos': 0}. Best is trial 0 with value: 80.0.


Returning generate_prediction = ChainOfThought(OptimizedDepartmentSuggestion(context, user_query -> department
    instructions='Analyze the user query, taking into consideration contextual cues and historical context, to \nprogrammatically suggest the optimal department for addressing the query proficiently.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'may contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    user_query = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'User Query:', 'desc': '${user_query}'})
    department = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Department:', 'desc': '${department}'})
)) from continue_program


In [83]:
gpt4o.inspect_history(n=1)




Predict the best department to help with a new user query based on the context.

---

Follow the following format.

Context: may contain relevant facts

User Query: ${user_query}

Reasoning: Let's think step by step in order to ${produce the department}. We ...

Department: ${department}

---

Context:
[1] «Fraud Investigation Unit:
Investigates suspected insurance fraud cases.
Works closely with law enforcement and other agencies.
Provides support for both internal and external fraud matters.»
[2] «Securities Legal Team:
Handles legal matters related to securities laws and regulations.
Provides advice on compliance with securities regulations and standards.
Manages disputes and litigation involving securities issues.»
[3] «Insurance Fraud Prevention Team:
Investigates and addresses instances of insurance fraud.
Provides support for fraud prevention and detection efforts.
Works with law enforcement and other agencies to combat insurance fraud.»
[4] «Securities Litigation Team:
Handl

"\n\n\nPredict the best department to help with a new user query based on the context.\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nUser Query: ${user_query}\n\nReasoning: Let's think step by step in order to ${produce the department}. We ...\n\nDepartment: ${department}\n\n---\n\nContext:\n[1] «Fraud Investigation Unit:\nInvestigates suspected insurance fraud cases.\nWorks closely with law enforcement and other agencies.\nProvides support for both internal and external fraud matters.»\n[2] «Securities Legal Team:\nHandles legal matters related to securities laws and regulations.\nProvides advice on compliance with securities regulations and standards.\nManages disputes and litigation involving securities issues.»\n[3] «Insurance Fraud Prevention Team:\nInvestigates and addresses instances of insurance fraud.\nProvides support for fraud prevention and detection efforts.\nWorks with law enforcement and other agencies to combat insurance fraud.»\n[4] «

In [136]:
evaluate(third_compiled_bot, metric=metric)

Average Metric: 13 / 15  (86.7): 100%|██████████| 15/15 [00:00<00:00, 26.55it/s]


,user_query,department,example_answer,pred_answer,answer_exact_match
0,we need help with a patent infringement case,Intellectual Property Group,Intellectual Property Group,Intellectual Property Group,✔️ [True]
1,we're dealing with a class action lawsuit,Class Action Defense Team,Class Action Defense Team,Class Action Litigation Team,False
2,we need assistance with a corporate restructuring,Corporate Transactions Legal Team,Corporate Transactions Legal Team,Corporate Transactions Legal Team,✔️ [True]
3,we're facing a crisis situation and need legal support,Crisis Management Legal Team,Crisis Management Legal Team,Crisis Management Legal Team,✔️ [True]
4,we need help with e-discovery for an upcoming case,E-Discovery Legal Team,E-Discovery Legal Team,E-Discovery Legal Team,✔️ [True]


(86.67,
 [(Example({'user_query': 'we need help with a patent infringement case', 'department': 'Intellectual Property Group', 'answer': 'Intellectual Property Group'}) (input_keys={'context', 'user_query'}),
   Prediction(
       answer='Intellectual Property Group'
   ),
   True),
  (Example({'user_query': "we're dealing with a class action lawsuit", 'department': 'Class Action Defense Team', 'answer': 'Class Action Defense Team'}) (input_keys={'context', 'user_query'}),
   Prediction(
       answer='Class Action Litigation Team'
   ),
   False),
  (Example({'user_query': 'we need assistance with a corporate restructuring', 'department': 'Corporate Transactions Legal Team', 'answer': 'Corporate Transactions Legal Team'}) (input_keys={'context', 'user_query'}),
   Prediction(
       answer='Corporate Transactions Legal Team'
   ),
   True),
  (Example({'user_query': "we're facing a crisis situation and need legal support", 'department': 'Crisis Management Legal Team', 'answer': 'Crisi

In [187]:
len(trainset)

20

In [188]:
from dspy.teleprompt import KNNFewShot, LabeledFewShot

knn_teleprompter = KNNFewShot(k=10, trainset=trainset)
compiled_knn = knn_teleprompter.compile(ConciergeBot(), trainset=trainset)

In [189]:
example = devset[10]
pred = compiled_knn(user_query = example.user_query)
print("Question: ", example.user_query)
print("Expected answer: ", example.department)
print("Prediction: ", pred.answer)

 40%|████      | 4/10 [00:00<00:00, 26.85it/s]


Question:  we need assistance with a healthcare compliance matter
Expected answer:  Healthcare Compliance Legal Team
Prediction:  Healthcare Compliance Legal Team


In [190]:
gpt4o.inspect_history(1)




Predict the best department to help with a new user query based on the context.

---

User Query: our company is being sued for alleged discrimination
Department: Employment Disputes

User Query: we have a dispute with a vendor regarding a contract
Department: Contractual Disputes Team

User Query: we are dealing with a challenge related to intellectual property
Department: Intellectual Property Group

User Query: i have a legal case that requires representation in court
Department: Field Legal

---

Follow the following format.

Context: may contain relevant facts

User Query: ${user_query}

Reasoning: Let's think step by step in order to ${produce the department}. We ...

Department: ${department}

---

Context:
[1] «Employment Benefits Legal Team:
Handles legal issues related to employee benefits and compensation.
Provides advice on retirement plans, health benefits, and compliance.
Manages disputes and litigation involving employee benefits.»
[2] «Employment Disputes:
Handles le

"\n\n\nPredict the best department to help with a new user query based on the context.\n\n---\n\nUser Query: our company is being sued for alleged discrimination\nDepartment: Employment Disputes\n\nUser Query: we have a dispute with a vendor regarding a contract\nDepartment: Contractual Disputes Team\n\nUser Query: we are dealing with a challenge related to intellectual property\nDepartment: Intellectual Property Group\n\nUser Query: i have a legal case that requires representation in court\nDepartment: Field Legal\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nUser Query: ${user_query}\n\nReasoning: Let's think step by step in order to ${produce the department}. We ...\n\nDepartment: ${department}\n\n---\n\nContext:\n[1] «Employment Benefits Legal Team:\nHandles legal issues related to employee benefits and compensation.\nProvides advice on retirement plans, health benefits, and compliance.\nManages disputes and litigation involving employee benefits.

In [191]:
evaluate(compiled_knn, metric=metric)

  0%|          | 0/15 [00:00<?, ?it/s]


 40%|████      | 4/10 [00:00<00:00, 27.03it/s]


Average Metric: 1 / 1  (100.0):   7%|▋         | 1/15 [00:00<00:02,  5.08it/s]


 40%|████      | 4/10 [00:00<00:00, 26.32it/s]


Average Metric: 1 / 2  (50.0):  13%|█▎        | 2/15 [00:00<00:02,  4.67it/s] 

...

Average Metric: 11 / 14  (78.6):  93%|█████████▎| 14/15 [00:02<00:00,  4.94it/s]


 40%|████      | 4/10 [00:00<00:00, 26.49it/s]


Average Metric: 11 / 15  (73.3): 100%|██████████| 15/15 [00:03<00:00,  4.91it/s]


,user_query,department,example_answer,pred_answer,reasoning,answer_exact_match
0,we need help with a patent infringement case,Intellectual Property Group,Intellectual Property Group,Intellectual Property Group,"produce the department. We are dealing with a case involving intellectual property rights, specifically patent infringement.",✔️ [True]
1,we're dealing with a class action lawsuit,Class Action Defense Team,Class Action Defense Team,Class Action Litigation Team,"produce the department. We are facing a class action lawsuit, which involves complex legal issues, defense against multiple plaintiffs, and compliance with class action laws...",False
2,we need assistance with a corporate restructuring,Corporate Transactions Legal Team,Corporate Transactions Legal Team,Mergers & Acquisitions Legal Team,"produce the department. We are dealing with a corporate restructuring, which involves legal matters related to corporate transactions, mergers, acquisitions, and compliance issues. The department...",False
3,we're facing a crisis situation and need legal support,Crisis Management Legal Team,Crisis Management Legal Team,Crisis Management Legal Team,"produce the department. Given that the user is facing a crisis situation and needs legal support, the optimal department to address this query proficiently would...",✔️ [True]
4,we need help with e-discovery for an upcoming case,E-Discovery Legal Team,E-Discovery Legal Team,E-Discovery Legal Team,"produce the department. We need assistance with e-discovery for an upcoming case, which involves managing electronic discovery processes, data collection, review, and production. This requires...",✔️ [True]


73.33

In [192]:
compiled_knn.save("compiled_bot_knnv2.json")

In [194]:
compiled_knn

generate_prediction = ChainOfThought(OptimizedDepartmentSuggestion(context, user_query -> department
    instructions='Analyze the user query, taking into consideration contextual cues and historical context, to \nprogrammatically suggest the optimal department for addressing the query proficiently.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'may contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    user_query = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'User Query:', 'desc': '${user_query}'})
    department = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Department:', 'desc': '${department}'})
))

In [186]:
gpt4o.inspect_history(1)




Predict the best department to help with a new user query based on the context.

---

User Query: our company is being sued for alleged discrimination
Department: Employment Disputes

User Query: we have a dispute with a vendor regarding a contract
Department: Contractual Disputes Team

User Query: we are dealing with a challenge related to intellectual property
Department: Intellectual Property Group

User Query: i have a legal case that requires representation in court
Department: Field Legal

---

Follow the following format.

Context: may contain relevant facts

User Query: ${user_query}

Reasoning: Let's think step by step in order to ${produce the department}. We ...

Department: ${department}

---

Context:
[1] «Employment Benefits Legal Team:
Handles legal issues related to employee benefits and compensation.
Provides advice on retirement plans, health benefits, and compliance.
Manages disputes and litigation involving employee benefits.»
[2] «Employment Disputes:
Handles le

"\n\n\nPredict the best department to help with a new user query based on the context.\n\n---\n\nUser Query: our company is being sued for alleged discrimination\nDepartment: Employment Disputes\n\nUser Query: we have a dispute with a vendor regarding a contract\nDepartment: Contractual Disputes Team\n\nUser Query: we are dealing with a challenge related to intellectual property\nDepartment: Intellectual Property Group\n\nUser Query: i have a legal case that requires representation in court\nDepartment: Field Legal\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nUser Query: ${user_query}\n\nReasoning: Let's think step by step in order to ${produce the department}. We ...\n\nDepartment: ${department}\n\n---\n\nContext:\n[1] «Employment Benefits Legal Team:\nHandles legal issues related to employee benefits and compensation.\nProvides advice on retirement plans, health benefits, and compliance.\nManages disputes and litigation involving employee benefits.

In [182]:
compiled_knn.save("compined_bot_knnv2.json")

In [140]:
evaluate_on_dev = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

# Evaluate the `compiled_knn` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match

evaluate_on_dev(compiled_knn, metric)

  0%|          | 0/15 [00:00<?, ?it/s]


 40%|████      | 4/10 [00:00<00:00, 27.78it/s]


Average Metric: 1 / 1  (100.0):   7%|▋         | 1/15 [00:00<00:02,  5.18it/s]


 40%|████      | 4/10 [00:00<00:00, 27.40it/s]


Average Metric: 1 / 2  (50.0):  13%|█▎        | 2/15 [00:00<00:02,  5.15it/s] 

...

Average Metric: 12 / 14  (85.7):  93%|█████████▎| 14/15 [00:02<00:00,  5.50it/s]


 40%|████      | 4/10 [00:00<00:00, 29.63it/s]


Average Metric: 12 / 15  (80.0): 100%|██████████| 15/15 [00:02<00:00,  5.30it/s]


,user_query,department,example_answer,pred_answer,answer_exact_match
0,we need help with a patent infringement case,Intellectual Property Group,Intellectual Property Group,Intellectual Property Group,✔️ [True]
1,we're dealing with a class action lawsuit,Class Action Defense Team,Class Action Defense Team,Class Action Litigation Team,False
2,we need assistance with a corporate restructuring,Corporate Transactions Legal Team,Corporate Transactions Legal Team,Corporate Transactions Legal Team,✔️ [True]
3,we're facing a crisis situation and need legal support,Crisis Management Legal Team,Crisis Management Legal Team,Crisis Management Legal Team,✔️ [True]
4,we need help with e-discovery for an upcoming case,E-Discovery Legal Team,E-Discovery Legal Team,E-Discovery Legal Team,✔️ [True]


80.0

In [195]:
compiled_knn.save("compiled_bot_knn.json")

In [198]:
from dspy.retrieve.faiss_rm import FaissRM
from dsp.modules.sentence_vectorizer import OpenAIVectorizer

In [200]:
frm = FaissRM(document_chunks=class_descriptions)

In [216]:
from src.dspy.utils import extract_labels_from_strings


class OptimizedDepartmentSuggestion(dspy.Signature):
    __doc__=f"""Given a user query and available departments, predict the 10 best matching departments."""
    
    context = dspy.InputField(desc="may contain relevant facts")
    options = dspy.InputField(
        prefix="Options:",
        desc="List of comma-separated options to choose from",
        format=lambda x: ", ".join(x) if isinstance(x, list) else x,
    )
    user_query = dspy.InputField(
        prefix="User Query:"
    )
    output = dspy.OutputField(
        prefix="Departments:",
        desc="list of comma-separated department names",
        format=lambda x: ", ".join(x) if isinstance(x, list) else x,
    )
    
    
class ConciergeBot(dspy.Module):
    def __init__(self, num_passages=5, class_labels = []):
        super().__init__()
        
        self.rm = dspy.Retrieve(k=num_passages)
        self.generate_prediction = dspy.ChainOfThought(OptimizedDepartmentSuggestion)
        self.valid_labels = class_labels

    def forward(self, user_query):
        parsed_outputs = set()
        context = self.rm(user_query).passages
        result = self.generate_prediction(context=context, options=self.valid_labels, user_query=user_query)
        parsed_outputs.update(
            extract_labels_from_strings(result.output, do_lower=False, strip_punct=False)
        )
        return dspy.Prediction(
            answer=result.output,
            reasoning=result.rationale,
            )

In [212]:
base_bot = ConciergeBot(
    num_passages=10,
    class_labels=class_labels,
)

In [213]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

gpt4T = dspy.OpenAI(model='gpt-4o', max_tokens=350, model_type='chat')

bootstrap_optimizer = BootstrapFewShotWithRandomSearch(
    max_bootstrapped_demos=8,
    max_labeled_demos=8,
    num_candidate_programs=10,
    num_threads=8,
    metric=metric,
    teacher_settings=dict(lm=gpt4T))


cot_fewshot = bootstrap_optimizer.compile(ConciergeBot(), trainset=trainset, valset=devset)
cot_fewshot.save("compiled_bot_cot_fewshotv2.json")

  0%|          | 0/15 [00:00<?, ?it/s]

Average Metric: 9 / 15  (60.0): 100%|██████████| 15/15 [00:00<00:00, 43.10it/s]


In [214]:
cot_fewshot.dump_state()

{'rm': {'k': 5},
 'generate_prediction': {'lm': None,
  'traces': [],
  'train': [],
  'demos': [Example({'augmented': True, 'context': ['Employment Benefits Legal Team:\nHandles legal issues related to employee benefits and compensation.\nProvides advice on retirement plans, health benefits, and compliance.\nManages disputes and litigation involving employee benefits.', 'Employment Disputes:\nHandles legal issues concerning current or former employees pursuing legal action against the company.\nCases include wrongful termination, payment-related disputes, or contractual matters.\nDoes not handle discrimination accusations.', 'Employment Compliance Legal Team:\nEnsures compliance with employment laws and regulations.\nConducts internal audits and reviews of employment practices.\nProvides guidance on employment compliance issues.', 'Employment Law Compliance Team:\nEnsures compliance with employment laws and regulations.\nConducts internal audits and reviews of employment practices.\nP

In [217]:
from dspy.teleprompt import KNNFewShot

knn_teleprompter = KNNFewShot(7, trainset)
compiled_knn = knn_teleprompter.compile(ConciergeBot(class_labels=class_labels), trainset=trainset)

In [219]:
example = devset[0]
pred = compiled_knn(user_query = example.user_query)
print("Question: ", example.user_query)
print("Expected answer: ", example.answer)
print("Prediction: ", pred.answer)

  0%|          | 0/7 [00:00<?, ?it/s]

 57%|█████▋    | 4/7 [00:05<00:04,  1.36s/it]


Question:  we need help with a patent infringement case
Expected answer:  Intellectual Property Group
Prediction:  Intellectual Property Group, Intellectual Property Litigation Team, Technology and Innovation Legal Team, White Collar Defense Legal Team, Litigation Support Team

Top 10 Predicted Matching Departments:
1. Intellectual Property Group
2. Intellectual Property Litigation Team
3. Technology and Innovation Legal Team
4. White Collar Defense Legal Team
5. Litigation Support Team


In [220]:
compiled_knn.save("compiled_bot_cot_knn.json")